In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import nltk

In [14]:
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [42]:
df = pd.read_csv('C:\\Users\\Gagan\\Desktop\\Ml Models\\Reviews.csv\\Reviews.csv')
df = df.head(10000)
print(df.shape)
df

(10000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ..."
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...


In [43]:
ex = df['Text'][10]
print(ex)

I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [44]:
Model = f'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(Model)
model = AutoModelForSequenceClassification.from_pretrained(Model)

In [45]:
tokenizer(ex,return_tensors="pt")

{'input_ids': tensor([[    0,   100,   218,    75,   216,   114,    24,    18,     5,   740,
         33028,    50,     5,  3055, 27224,    50,    95,     5,  2216,  4069,
             9,  7075,     6,    53,     5, 24253,     9,    42,  2131,  8929,
           817,    24,    65,     9,    10,   761,   328,  1437,   166,  2738,
            62,    10,  7304,   683,    15,    10,  1805,    52,    58,    15,
             8,  1146,    24,   124,   184,    19,   201,     8,    58,  4940,
         12315,   409,   328,  1437,   520,    52,  5426,    14,    52,  1622,
          1705,    75,   465,    24,  4558,    11,    84,   343,    52,    58,
         29673,  4567, 49069,  3809,  1589, 49007,  3809, 48709,  5975,     6,
           142,     9,     5,  8375,     9,     5,  2888,     6,    52,    33,
            10,   403,     9,     5,  8929,     8,    32, 30754,   142,     9,
            24, 49069,  3809,  1589, 49007,  3809, 48709,  1106,    47,   657,
          2131,  8929,  7586,   100,  

In [46]:
e_text = tokenizer(ex,return_tensors="pt")
output = model(**e_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.019134127, 'roberta_neu': 0.071044475, 'roberta_pos': 0.9098214}


In [47]:
def polarity_scores_roberta(ex):
    e_text = tokenizer(ex,return_tensors="pt")
    output = model(**e_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [48]:
res = {}
for i,row in tqdm(df.iterrows(),total=len(df)):
    try:
        text = row['Text']
        myid = row['Id']
        res[myid] = polarity_scores_roberta(text)
    except RuntimeError:
        print(f'Broke for id {myid}')

  0%|          | 0/10000 [00:00<?, ?it/s]

Broke for id 83
Broke for id 187
Broke for id 529
Broke for id 540
Broke for id 746
Broke for id 863
Broke for id 1053
Broke for id 1070
Broke for id 1156
Broke for id 1321
Broke for id 1375
Broke for id 1498
Broke for id 1575
Broke for id 1796
Broke for id 1826
Broke for id 2169
Broke for id 2248
Broke for id 2476
Broke for id 2492
Broke for id 2584
Broke for id 2610
Broke for id 2897
Broke for id 2898
Broke for id 2902
Broke for id 2928
Broke for id 2942
Broke for id 2944
Broke for id 2947
Broke for id 2948
Broke for id 3022
Broke for id 3023
Broke for id 3025
Broke for id 3306
Broke for id 3788
Broke for id 3969
Broke for id 4107
Broke for id 4110
Broke for id 4307
Broke for id 4316
Broke for id 4408
Broke for id 4483
Broke for id 4512
Broke for id 4553
Broke for id 4583
Broke for id 5040
Broke for id 5182
Broke for id 5192
Broke for id 5365
Broke for id 5442
Broke for id 5709
Broke for id 5998
Broke for id 6003
Broke for id 6007
Broke for id 6106
Broke for id 6234
Broke for id 6558

In [60]:
res

{1: {'roberta_neg': 0.009624249,
  'roberta_neu': 0.049980447,
  'roberta_pos': 0.9403953},
 2: {'roberta_neg': 0.5089861,
  'roberta_neu': 0.45241356,
  'roberta_pos': 0.038600363},
 3: {'roberta_neg': 0.003228897,
  'roberta_neu': 0.09806738,
  'roberta_pos': 0.8987037},
 4: {'roberta_neg': 0.0022951285,
  'roberta_neu': 0.09021923,
  'roberta_pos': 0.90748566},
 5: {'roberta_neg': 0.0016347291,
  'roberta_neu': 0.01030247,
  'roberta_pos': 0.98806286},
 6: {'roberta_neg': 0.0061295927,
  'roberta_neu': 0.021795873,
  'roberta_pos': 0.97207457},
 7: {'roberta_neg': 0.0013047527,
  'roberta_neu': 0.007773783,
  'roberta_pos': 0.99092144},
 8: {'roberta_neg': 0.002029356,
  'roberta_neu': 0.0056852014,
  'roberta_pos': 0.9922854},
 9: {'roberta_neg': 0.0038735769,
  'roberta_neu': 0.094156265,
  'roberta_pos': 0.9019702},
 10: {'roberta_neg': 0.0031034567,
  'roberta_neu': 0.05591218,
  'roberta_pos': 0.9409843},
 11: {'roberta_neg': 0.019134127,
  'roberta_neu': 0.071044475,
  'robert

In [72]:
results = pd.DataFrame(res).T
results = results.reset_index().rename(columns={'index': 'id'})

results

,id,roberta_neg,roberta_neu,roberta_pos
0,1,0.009624,0.049980,0.940395
1,2,0.508986,0.452414,0.038600
2,3,0.003229,0.098067,0.898704
3,4,0.002295,0.090219,0.907486
4,5,0.001635,0.010302,0.988063
...,...,...,...,...
9912,9996,0.339595,0.407847,0.252558
9913,9997,0.010780,0.110858,0.878362
9914,9998,0.036277,0.215930,0.747794
9915,9999,0.011023,0.043542,0.945435
